In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline
import numpy as np

/home/adminserv/miniconda3/envs/albert_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn",device=1)


/home/adminserv/miniconda3/envs/albert_env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
import pandas as pd
df=pd.read_csv('./bertopic/results.csv')
df['Question'] = df['Question'].astype(str).apply(lambda x: x.lower())

In [3]:
df.head()

,Question,doclen,Topic,Topic_Label
0,"like everyone else (here in u.s), i work with ...",125,-1,General
1,hello dear's people. i have a fictief research...,80,6,Gun Safety Regulations
2,a lady buys goods worth 200 bucks from a shop ...,77,2,Drug Testing and Probability
3,i am turning 25 in about a month and am curren...,73,1,Job Prospects by Major
4,i'm a 34 years old married to a woman. i had a...,73,4,Dating and Communication


In [4]:
topic_id_name_map = df[['Topic', 'Topic_Label']].drop_duplicates().set_index('Topic')['Topic_Label'].to_dict()
print(topic_id_name_map)

{-1: 'General', 6: 'Gun Safety Regulations', 2: 'Drug Testing and Probability', 1: 'Job Prospects by Major', 4: 'Dating and Communication', 3: 'Universe and Energy', 5: 'Snapchat Account Recovery', 0: 'Cambodia Earthquake and Historical Battles'}


In [5]:
grouped = df.groupby('Topic')

In [8]:
for topic_id, topic_name in topic_id_name_map.items():
    topic_df = grouped.get_group(topic_id).drop(columns='Topic')
    print(f'Topic: {topic_name}')
    print(topic_df.head())
    print('\n\n')

Topic: General
                                             Question  doclen Topic_Label
0   like everyone else (here in u.s), i work with ...     125     General
5   my employer has told me that we can not accept...      72     General
10  i feel like i'm getting over my crush. this fe...      67     General
16  i belong to obc. my ctc is 7 lacs per annum bu...      64     General
17  i have 2.6 years of it experience. i got marri...      64     General



Topic: Gun Safety Regulations
                                              Question  doclen  \
1    hello dear's people. i have a fictief research...      80   
26   i am supposed to wear contacts but i haven't f...      62   
299  if two automatic weapons ( uzi ) were pointed ...      47   
387  i am trying to design a professional poster an...      45   
588  if local detectives show up at your door and s...      42   

                Topic_Label  
1    Gun Safety Regulations  
26   Gun Safety Regulations  
299  Gun Safety Regul

In [17]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [11]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration

# Specify device for the second GPU
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# Load tokenizer and model, then move model to the specified device
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn").to(device)


# Function to split text into chunks suitable for summarization
def chunk_text(text, max_length=1024):
    sentences = text.split('. ')
    chunks = []
    current_chunk = ""
    
    for sentence in sentences:
        # Clean sentence and avoid punctuation that could add noise
        sentence = sentence.strip()
        if len(current_chunk) + len(sentence) > max_length:
            chunks.append(current_chunk)
            current_chunk = sentence
        else:
            current_chunk += " " + sentence
            
    if current_chunk:
        chunks.append(current_chunk)
        
    return chunks

# Summarization function that handles long texts by chunking
def summarize_text(text, max_tokens=200, min_length=25):
    summaries = []
    chunks = chunk_text(text, max_length=4096)
    for idx, chunk in enumerate(chunks):
        inputs = tokenizer(chunk, return_tensors="pt", truncation=True, max_length=1024).to(device)
        summary_ids = model.generate(inputs["input_ids"], max_new_tokens=max_tokens, min_length=min_length, do_sample=False)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)
    
    # Join all summaries from each chunk
    full_summary = " \n\n\n".join(summaries)
    return full_summary




/home/adminserv/miniconda3/envs/albert_env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
with open('topic_summary_bart.txt', 'w',encoding="utf-8") as f:
    f.write('Topic Summary\n\n')
    for key,value in topic_id_name_map.items():
        f.write(f'\nTopic: {value}\n')
        topic_df = grouped.get_group(key).drop(columns='Topic')
        f.write(f'\nTotal number of questions: {topic_df.shape[0]}\n')
        documents ="." .join(topic_df['Question'].values)
        f.write(f'\nTotal No of tokens:{len(documents.split())}')
        summary = summarize_text(documents)
        f.write(f"\n\nTotal Tokens in summary:{len(summary.split())}\n")
        print("Summary:", summary)
        f.write(f'\n\nSummary: {summary}\n')

Summary: i'm a human rights advocate and lesbian, my views are liberal and i by no means preach my beliefs in my work place. When is it ok to state your opinions on an issue for example, if someone mentions that their next door neighbor, who happens to be a 12 year old boy, was playing with dolls and that "this is going down the wrong road and he is strange/weird" Is it ok for me to say that we shouldn't judge him for who he is? 


i like a guy, he is a very nice person, awesome intellect, but he is not that good looking and doesn't know how to talk i mean he bores me a lot i try to ignore his looks, but i am always disappointed because he doesn't even talk nicely or make me happy he's not even romantic what should i do?my friend while working was met with an accident as he is working as a daily labourer and he was the sole bread earning individual of his family, there is no one to take care to help him cover his treatment needs right now. 


i married a girl but she was my parents cho